In [1]:
import numpy as np
import pandas as pd
import matplotlib.image as mpimg       
import matplotlib.pyplot as plt        
import matplotlib.patches as mpatches 
from skimage import measure         
import scipy.ndimage as ndi  
from os.path import exists, join
from os import listdir
import cv2
import sys

In [2]:
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = (6, 6) 

In [3]:
import os
images = os.listdir('sample_data/images')
images

FileNotFoundError: [Errno 2] No such file or directory: 'sample_data/images'

In [ ]:
import os
from skimage import io

In [ ]:
def input_img (img_file):
    img_dir = 'sample_data/images'
#     user_input = input('Enter file name')
    if os.path.exists(img_file):
        img = io.imread(img_file)
        return img
    else:
        return f'file not found'

In [ ]:
img_dir = 'sample_data/images'
user_input = input('Enter file name : ')

img_file_path = os.path.join(img_dir, user_input)
img = input_img(img_file_path)
io.imshow(img)

In [ ]:
img.shape

In [ ]:
from skimage import data
from skimage.exposure import histogram

In [ ]:
import tensorflow as tf

# image_path = img
def loadImages(img):
    '''Put files into lists and return them as one list with all images 
     in the folder'''
    image_files = sorted([os.path.join(path, 'train', file)
                          for file in os.listdir(path + "/train")
                          if file.endswith('.png')])
    return image_files

In [ ]:
# Display image
def display_one(img, title1 = "Original"):
    plt.imshow(img), plt.title(title1)
    plt.show()


In [ ]:
# Preprocessing
def processing(data):
    
    # Reading 3 images to work
    img = [cv2.imread(i, cv2.IMREAD_UNCHANGED) for i in data[:3]]
    try:
        print('Original size',img[0].shape)
    except AttributeError:
        print("shape not found")
   
    # --------------------------------
    # setting dim of the resize
    height = 220
    width = 220
    dim = (width, height)
    res_img = []
    for i in range(len(img)):
        res = cv2.resize(img[i], dim, interpolation=cv2.INTER_LINEAR)
        res_img.append(res)

    # Checcking the size
    try:
        print('RESIZED', res_img[1].shape)
    except AttributeError:
        print("shape not found")
    
    
    # Visualizing one of the images in the array
    original = res_img[1]
    display_one(original)
    # ----------------------------------
    # Remove noise
    # Using Gaussian Blur
    no_noise = []
    for i in range(len(res_img)):
        blur = cv2.GaussianBlur(res_img[i], (5, 5), 0)
        no_noise.append(blur)


    image = no_noise[1]
    display(original, image, 'Original', 'Blured')
    #---------------------------------
    # Segmentation
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Displaying segmented images
    display(original, thresh, 'Original', 'Segmented')
    # Further noise removal (Morphology)
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # sure background area
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    #Displaying segmented back ground
    display(original, sure_bg, 'Original', 'Segmented Background')

    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1

    # Now, mark the region of unknown with zero
    markers[unknown == 255] = 0

    markers = cv2.watershed(image, markers)
    image[markers == -1] = [255, 0, 0]

    # Displaying markers on the image
    display(original, markers, 'Original', 'Marked')

In [ ]:
def main():
    # calling global variable
    global image_path
    '''The var Dataset is a list with all images in the folder '''
    dataset = loadImages(image_path)
    print('number of FILES in dir', len(dataset))
    print("--------------------------------")
    print(cv2.imread(dataset[0]).shape)
    print("List of files the first 3 in the folder:\n",dataset[:3])
    print("--------------------------------")
    
    # sending all the images to pre-processing
    processing(dataset)
   
    #list files in directory
    #a = tf.gfile.ListDirectory('drive/My Drive/BiSeNet/dataset/train')
    #print(a)
  
main()